# 1. 池化层（汇聚层）

- 卷积对位置太过敏感，但又需要一定程度的平移不变性，所以有池化层
<img src="./pic/池化层.PNG" width=400 height=400>

## 二维最大池化
- 不像卷积层（核与输入做哈达玛积）。而是每次选取窗口中，最大的值

<table><tr>
    <td><img src="./pic/二维最大池化.PNG" width=300 height=300></td>
    <td><img src="./pic/二维交叉相关(卷积核).gif" width=200 height=200></td>
</tr></table>
<img src="./pic/二维最大池化例子1.PNG" width=400 height=400>
- 池化层允许输入发生小偏移，作用在卷积输出上，使输出值附近的小窗口的值都会出现。本质上是池化层对于像素偏移的容忍性


## 填充，步幅和多个通道
- 与卷积层不同，池化层不会做融合通道。所以输出通道数=输入通道数
<table><tr>
    <td><img src="./pic/填充，步幅和多个通道.PNG" width=300 height=300></td>
    <td><img src="./pic/有边界填充，跨步.gif" width=200 height=200></td>
</tr></table>

## 平均池化层
- 最大池化层：有明显的层次化信息
- 平均池化层：抹平信号强度,较柔和，不会有强烈抖动
<img src="./pic/平均池化层.PNG" width=300 height=300>


## 总结
- 池化层返回窗口中最大或平均值
- 缓解卷积层对位置的敏感性
- 同样有窗口大小、填充、和步幅作为超参数
- 没有可以学习的参数

# 2. 池化层的实现

## 实现池化层的正向传播

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [2]:
# 输入张量X，验证二维最大汇聚层的输出
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [3]:
# 验证平均汇聚层
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

## 填充和步幅

In [31]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))    # 批量大小为1，通道为1
X, X.shape

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]]]]),
 torch.Size([1, 1, 4, 4]))

- 深度学习框架中的步幅与汇聚窗口的大小相同
    - 看完当前窗口后，下一个窗口与当前窗口不重叠

In [27]:
pool2d = nn.MaxPool2d(3)    # 3x3的窗口，默认步幅等于窗口size（滑窗不重合）
pool2d(X)

tensor([[[[10.]]]])

- 填充和步幅可以手动设定

In [28]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

- 设定一个任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度

In [29]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(1, 1))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

## 池化层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总

In [38]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))    # 批量大小为1，通道为1
X = torch.cat((X, X + 1), 1)    # 要给X加一个通道，所以只能在通道的那一维添加，axis=1
X, X.shape



(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]],
 
          [[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]]]]),
 torch.Size([1, 2, 4, 4]))

In [39]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)
# 第一个通道和第二个通道

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

In [45]:
# Q：为什么不用stack而用cat：
# A：torch.stack会增加一个新轴，torch.cat只会在某个特定的轴上对元素进行拼接。
# 这里a【X】本身已经包含了batch_size和通道两个轴，在通道这个轴上进行的拼接，cat的参数是1
a = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))    # 批量大小为1，通道为1
b = torch.cat((a, a + 1), 1)    # 上例，轴=1
c =  torch.cat((a, a + 1), 0)    # 轴0，变成了2个batch，每个batch只有单通道
d = torch.stack((a, a + 1), 0)    # stack：升1维
b, b.shape, c, c.shape, d, d.shape

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]],
 
          [[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]]]]),
 torch.Size([1, 2, 4, 4]),
 tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]]],
 
 
         [[[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]]]]),
 torch.Size([2, 1, 4, 4]),
 tensor([[[[[ 0.,  1.,  2.,  3.],
            [ 4.,  5.,  6.,  7.],
            [ 8.,  9., 10., 11.],
            [12., 13., 14., 15.]]]],
 
 
 
         [[[[ 1.,  2.,  3.,  4.],
            [ 5.,  6.,  7.,  8.],
            [ 9., 10., 11., 12.],
            [13., 14., 15., 16.]]]]]),
 torch.Size([2, 1, 1, 4, 4]))

In [46]:
pool2d(c)

tensor([[[[ 5.,  7.],
          [13., 15.]]],


        [[[ 6.,  8.],
          [14., 16.]]]])

- Q：为什么现在池化层用的越来越少？
- A：池化的目的之一是减少计算，但卷积层strides=2也能减少计算。
    - 之二是因为我们会对数据做扰动操作（旋转，缩放）使得神经网络可以看到数据本身发生很多变化。使得数据在卷积层不会过拟合到某个具体的位置
    - 所以就淡化的池化层的作用